# 케글 코로나 API 데이터셋 사용하기

### 1.pip install kaggle --upgrade

### 2. 케글 회원가입 
    
    https://www.kaggle.com/<자기이름>/account 혹은 my Account에 접속후
    하단의 create New API Token 클릭

### 3. 아래의 경로로 kaggel.json파일 이동 (mv 사용) 

    window user  -> C:\Users\<Windows-username>\.kaggle\kaggle.json
    Mac os user   ->   ~/.kaggle/kaggle.json


### 4. 케글 covid-19 south korea data 들고오기
    
    터미널에서 입력 -> kaggle datasets download -d kimjihoo/coronavirusdataset
                   unzip coronavirusdataset.zip

# 다운 받은 covid-19 data는 cumsum 처리가 되어있다
## 월별로 확진자수를 확인하기 위해서 데이터를 처리해 보자!

In [1]:
import pandas as pd

In [2]:
#영어로 되어있는 지역이름을 바꾸는 과정
covid_df = pd.read_csv('/Users/youngyong/TimeProvince.csv')
covid_df = covid_df.sort_values(['province','date']).reset_index()
covid_df = covid_df.drop('index', axis=1)

cp_covid_df = covid_df.copy()

def rt_rigion(i):
    return cp_covid_df.loc[i,'province']

for i in range(len(covid_df['province'])):
    if rt_rigion(i) == 'Seoul': cp_covid_df.loc[i,'province'] = '서울'
    elif rt_rigion(i) == 'Busan': cp_covid_df.loc[i,'province'] = '부산'
    elif rt_rigion(i) == 'Daegu': cp_covid_df.loc[i,'province'] = '대구'
    elif rt_rigion(i) == 'Incheon': cp_covid_df.loc[i,'province'] = '인천'
    elif rt_rigion(i) == 'Gwangju': cp_covid_df.loc[i,'province'] = '광주'
    elif rt_rigion(i) == 'Daejeon': cp_covid_df.loc[i,'province'] = '대전'
    elif rt_rigion(i) == 'Ulsan': cp_covid_df.loc[i,'province'] = '울산'    
    elif rt_rigion(i) == 'Sejong': cp_covid_df.loc[i,'province'] = '세종'
    elif rt_rigion(i) == 'Gyeonggi-do': cp_covid_df.loc[i,'province'] = '경기'
    elif rt_rigion(i) == 'Gangwon-do': cp_covid_df.loc[i,'province'] = '강원'
    elif rt_rigion(i) == 'Chungcheongbuk-do': cp_covid_df.loc[i,'province'] = '충북'
    elif rt_rigion(i) == 'Chungcheongnam-do': cp_covid_df.loc[i,'province'] = '충남'
    elif rt_rigion(i) == 'Jeollabuk-do': cp_covid_df.loc[i,'province'] = '전북'
    elif rt_rigion(i) == 'Jeollanam-do': cp_covid_df.loc[i,'province'] = '전남'
    elif rt_rigion(i) == 'Gyeongsangbuk-do': cp_covid_df.loc[i,'province'] = '경북'
    elif rt_rigion(i) == 'Gyeongsangnam-do': cp_covid_df.loc[i,'province'] = '경남'
    elif rt_rigion(i) == 'Jeju-do': cp_covid_df.loc[i,'province'] = '제주'
    

    
covid_df.head()

,date,time,province,confirmed,released,deceased
0,2020-01-20,16,Busan,0,0,0
1,2020-01-21,16,Busan,0,0,0
2,2020-01-22,16,Busan,0,0,0
3,2020-01-23,16,Busan,0,0,0
4,2020-01-24,16,Busan,0,0,0


In [3]:
cp_covid_df.head()

,date,time,province,confirmed,released,deceased
0,2020-01-20,16,부산,0,0,0
1,2020-01-21,16,부산,0,0,0
2,2020-01-22,16,부산,0,0,0
3,2020-01-23,16,부산,0,0,0
4,2020-01-24,16,부산,0,0,0


In [33]:
# 각 지역마다 데이터 갯수 163
# 일자별 지역별 코로나 확진자 수를 구하기위한 과정

#지역의 확진사 수를 받기위한 배열


# cumsum처리 되어있는 확진자수와 격리해제된 사람들을 역으로 다시 일별 데이터로 수정하는 함수
def reverse_cumsum(variable):
    covid_tmp_num = []

    # 각지역 딕셔너리 생성
    rigion_covid_num = dict()
    for i in cp_covid_df['province'].unique(): rigion_covid_num[i] =''


    # 지역별로 탐색
    for i in rigion_covid_num.keys():
        tmp_df = cp_covid_df[cp_covid_df['province']==i]
        tmp_df = tmp_df.reset_index()

        for j in range(len(tmp_df)):
            if j ==162: 
                covid_tmp_num.append(0)
                break
            else : covid_tmp_num.append(tmp_df.loc[j+1,variable] - tmp_df.loc[j,variable])


        rigion_covid_num[i] = covid_tmp_num
        covid_tmp_num=[]
    return rigion_covid_num

    

day_confirm = pd.DataFrame(reverse_cumsum('confirmed'))
day_release = pd.DataFrame(reverse_cumsum('released'))

day_confirm['date'] = cp_covid_df['date']
day_release['date'] = cp_covid_df['date']

day_confirm= pd.melt(day_confirm,id_vars = 'date')
day_release= pd.melt(day_release,id_vars = 'date')

day_confirm.columns = ['date','province','day_confirmed']
day_release.columns = ['date','province','day_release']

final_covid_df = pd.merge(day_confirm,day_release,on=['date','province'],how='inner')


final_covid_df.head()  

,date,province,day_confirmed,day_release
0,2020-01-20,부산,0,0
1,2020-01-21,부산,0,0
2,2020-01-22,부산,0,0
3,2020-01-23,부산,0,0
4,2020-01-24,부산,0,0


In [34]:
def grap_year(data):
    data = str(data)
    return int(data[:4])

def grap_month(data):
    data = str(data)
    return int(data[5:])

#월별 그룹화를 위한 일자 처리
for i in range(len(covid_df['date'])):
    final_covid_df.loc[i,'date'] = str(final_covid_df.loc[i]['date'])[0:7]


sum_covid_df = final_covid_df.groupby(['date','province']).sum().reset_index()

sum_covid_df['year'] = sum_covid_df['date'].apply(lambda x: grap_year(x))
sum_covid_df['month'] =sum_covid_df['date'].apply(lambda x: grap_month(x))
sum_covid_df = sum_covid_df.drop('date',axis=1)


sum_covid_df.head(100)

,province,day_confirmed,day_release,year,month
0,강원,0,0,2020,1
1,경기,6,0,2020,1
2,경남,0,0,2020,1
3,경북,0,0,2020,1
4,광주,0,0,2020,1
...,...,...,...,...,...
95,울산,3,5,2020,6
96,인천,117,192,2020,6
97,전남,4,2,2020,6
98,전북,6,2,2020,6


In [35]:
sum_covid_df = sum_covid_df[['province','day_confirmed','year','month']]
sum_covid_df

,province,day_confirmed,year,month
0,강원,0,2020,1
1,경기,6,2020,1
2,경남,0,2020,1
3,경북,0,2020,1
4,광주,0,2020,1
...,...,...,...,...
97,전남,4,2020,6
98,전북,6,2020,6
99,제주,4,2020,6
100,충남,21,2020,6


코로나 확진자수 7월  

7월  
강원 = 7    
경기 = 291    
경남 = 24   
경북=7    
광주 = 147   
대구 = 16   
대전 = 38   
부산 = 15    
서울 = 253    
세종 = 0    
울산 =3    
인천 = 26    
전남 = 11     
전북 = 12    
제주 = 5    
충남 =19    
충북 = 8    


 

 


In [37]:
province = []
num=[7,291,24,7,147,16,38,15,253,0,3,26,11,12,5,19,8]

for x,y in zip(sum_covid_df['province'].unique(), num):
    province.append([x,y])
province

[['강원', 7],
 ['경기', 291],
 ['경남', 24],
 ['경북', 7],
 ['광주', 147],
 ['대구', 16],
 ['대전', 38],
 ['부산', 15],
 ['서울', 253],
 ['세종', 0],
 ['울산', 3],
 ['인천', 26],
 ['전남', 11],
 ['전북', 12],
 ['제주', 5],
 ['충남', 19],
 ['충북', 8]]

In [39]:
a = pd.DataFrame(province)
a['year']=2020

month=[]
for i in range(17):month.append(7)
    
a['date'] = month
a.columns=['province','day_confirmed','year','month']
a

,province,day_confirmed,year,month
0,강원,7,2020,7
1,경기,291,2020,7
2,경남,24,2020,7
3,경북,7,2020,7
4,광주,147,2020,7
5,대구,16,2020,7
6,대전,38,2020,7
7,부산,15,2020,7
8,서울,253,2020,7
9,세종,0,2020,7


In [42]:
sum_covid_df = sum_covid_df.append(a)

In [43]:
sum_covid_df.to_csv('covid.csv',encoding='utf-8')